In [1]:
import json
import pandas as pd
import re
import pyproj

In [2]:
with open('WC_WIEN.json', encoding='utf-8') as json_file:
    zdroj_data = json.load(json_file)

In [3]:
data_list = []

for feature in zdroj_data['features']:
    object_id = feature['properties']['OBJECTID']
    address = feature['properties']['STRASSE']
    opening_hours = feature['properties']['OEFFNUNGSZEIT']
    barier = feature['properties']['ICON_TXT']
    longitude, latitude = feature["geometry"]["coordinates"]
    
    data_list.append([object_id, address, opening_hours, longitude, latitude, barier])


In [4]:

df = pd.DataFrame(data_list, columns=['OBJECTID', 'ADRESA', 'OTEVÍRACÍ DOBA', 'LONGITUDE', 'LATITUDE','BEZBAR'])

In [5]:
hours = df['OTEVÍRACÍ DOBA']
(hours.unique())

array(['00:00-24:00 Uhr', '10:00-18:00 Uhr', 'täglich 10-18 Uhr',
       'Di-Do und Sa-So 11:30 - 14:30 Fr 17:30 - 22:00',
       'Mo-Sa 10:00-19:00 Uhr So 9:00-19 Uhr', '11:00-23:00 Uhr',
       '9:00-21:00 Uhr', 'siehe Personalbetreuung',
       'während der Betriebszeiten', '06:00-23:00 Uhr',
       'Parköffnungszeiten', 'Mo-Fr 7:00-18:00 Uhr, Sa 8:00-18:00 Uhr',
       'Mo-Sa 7:00-18:00 Uhr', 'Lokalöffnungszeiten'], dtype=object)

In [6]:
unikatni_oteviraci_doby = ['00:00-24:00 Uhr', '10:00-18:00 Uhr', 'täglich 10-18 Uhr', '06:00-23:00 Uhr', 'Mo-Sa 7:00-18:00 Uhr', '9:00-21:00 Uhr']

df['PRACOVNÍ DNY'] = 'None'
df['VÍKEND'] = 'None'

for oteviraci_doba in unikatni_oteviraci_doby:
    df.loc[df['OTEVÍRACÍ DOBA'].str.contains(oteviraci_doba), 'PRACOVNÍ DNY'] = oteviraci_doba
    df.loc[df['OTEVÍRACÍ DOBA'].str.contains(oteviraci_doba), 'VÍKEND'] = oteviraci_doba



In [7]:
df.head()

,OBJECTID,ADRESA,OTEVÍRACÍ DOBA,LONGITUDE,LATITUDE,BEZBAR,PRACOVNÍ DNY,VÍKEND
0,300667,Rathauspark / Lichtenfelsgasse,00:00-24:00 Uhr,16.358469,48.209817,Barrierefrei,00:00-24:00 Uhr,00:00-24:00 Uhr
1,300668,SANIFAIR U-Bahn Station Karlsplatz,00:00-24:00 Uhr,16.368701,48.201239,Barrierefrei,00:00-24:00 Uhr,00:00-24:00 Uhr
2,300669,Badeteich Hirschstetten,00:00-24:00 Uhr,16.479620,48.242010,Barrierefrei,00:00-24:00 Uhr,00:00-24:00 Uhr
3,300670,Steinbauerpark / Steinbauergasse,00:00-24:00 Uhr,16.339466,48.181679,Barrierefrei,00:00-24:00 Uhr,00:00-24:00 Uhr
4,300671,Alois-Drasche-Park / Seisgasse,00:00-24:00 Uhr,16.367493,48.187047,Barrierefrei,00:00-24:00 Uhr,00:00-24:00 Uhr


In [8]:
def extract_hours_minutes(value):
    match = re.match(r'(\d+:\d+)-(\d+:\d+)', value)
    if match:
        return match.group(1), match.group(2)
    else:
        return None, None

df[['OTEVRENO_OD_PD', 'OTEVRENO_DO_PD']] = df['PRACOVNÍ DNY'].apply(lambda x: pd.Series(extract_hours_minutes(x)))
df[['OTEVRENO_OD_VIKEND', 'OTEVRENO_DO_VIKEND']] = df['VÍKEND'].apply(lambda x: pd.Series(extract_hours_minutes(x)))

df = df.drop(['MINUTY_OD_PD', 'MINUTY_DO_PD', 'MINUTY_OD_VIKEND', 'MINUTY_DO_VIKEND'], axis=1, errors='ignore')


In [9]:
data_dict = df.to_dict(orient='records')

with open('viden_ciste.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_dict, json_file, ensure_ascii=False)